In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import salem
import copy
import json

In [10]:
def hazard_ratio(theta, alpha, miu, v, concentration):
    z = (concentration-2.4) if (concentration - 2.4) > 0 else 0
    numerator = theta * np.log(z/(alpha)+1)
    denominator = 1 + np.exp(-(z-miu)/v)
    GEMM = np.exp(numerator/denominator)
    return GEMM 
def hazard_ratio_NCDLRI(concentration):
    if concentration < 2.4: # Lowest observed concentration in any of the 41 cohorts
        return 1
    else:
        return hazard_ratio(0.1430, 1.6, 15.5, 36.8, concentration)

In [67]:
fraction = pd.read_csv('comparison/country_classify.csv')
# fraction = pd.read_csv('C:/Users/15311/Desktop/country_classify.csv')

In [11]:
# file = open("comparison/Baseline.json", "r")
file = open("C:/Users/15311/Desktop/Baseline.json", "r")
baseline = file.read()
baseline = json.loads(baseline)
Country = list(baseline.keys())

## Calculate the total population of each country

In [15]:
file = open("comparison/Total_pop.json", "r")
# file = open("C:/Users/15311/Desktop/Total_pop_old.json", "r")
Total_pop = file.read()
Total_pop = json.loads(Total_pop)

## Allocate baseline mortality rate to each grid cell (Submit to cc part 1)

In [ ]:
# For ROW
A = ['agr','ene','ind','nrtr','rcor','rcoc','rcoo','shp','slv','wst']
for j in A:
    read_shp = gpd.read_file(r'output/output_wholeworld/wholeworld_' + j + '.' + 'shp')
    fraction['TotalPM25'] += read_shp['TotalPM25']

In [68]:
fraction['base_mort'] = [{} for i in range(len(fraction))]

In [70]:
fraction

,Unnamed: 0,AllCause,BasePM25,NH3,NOx,PNH4,PNO3,PSO4,SOA,SOx,TotalPM25,TotalPop,TotalPopD,geometry,area_fraction,base_mort
205,205,0.0,2.500696,0.000067,0.056172,0.000084,0.010858,0.000058,0.216152,8.941176e-15,0.242814,6172.600347,0.0,"POLYGON ((-158.125 21.5625, -158.046875 21.562...","{'USA': 0.8447576543578876, 'ROW': 0.155242345...",{}
206,206,0.0,2.500696,0.000071,0.095871,0.000088,0.018532,0.000061,0.354065,9.049820e-15,0.389382,17172.894133,0.0,"POLYGON ((-158.0859375 21.3125, -158.046875 21...",{'USA': 1.0},{}
207,207,0.0,2.500696,0.000071,0.097383,0.000088,0.018824,0.000061,0.397990,9.063032e-15,0.433499,11207.432410,0.0,"POLYGON ((-158.0859375 21.34375, -158.046875 2...",{'USA': 1.0},{}
208,208,0.0,2.500696,0.000067,0.042814,0.000084,0.008276,0.000058,0.149371,9.168077e-15,0.173394,6099.823062,0.0,"POLYGON ((-158.125 21.625, -157.96875 21.625, ...","{'USA': 0.3329359631475851, 'ROW': 0.667064036...",{}
209,209,0.0,2.500696,0.000072,0.095243,0.000089,0.018410,0.000062,0.351096,9.779626e-15,0.386535,37240.590657,0.0,"POLYGON ((-158.046875 21.3125, -158.0078125 21...",{'USA': 1.0},{}
210,210,0.0,2.500696,0.000071,0.097264,0.000089,0.018801,0.000062,0.399765,9.893337e-15,0.435528,13214.899134,0.0,"POLYGON ((-158.046875 21.34375, -158.0078125 2...","{'USA': 0.9441199705786774, 'ROW': 0.055880029...",{}
211,211,0.0,2.500696,0.000071,0.094490,0.000089,0.018265,0.000061,0.400988,9.845581e-15,0.436104,38753.139890,0.0,"POLYGON ((-158.046875 21.375, -158.0078125 21....","{'USA': 0.9766882949537582, 'ROW': 0.023311705...",{}
212,212,0.0,2.500696,0.000071,0.091036,0.000088,0.017597,0.000061,0.387806,9.897702e-15,0.422192,7348.331527,0.0,"POLYGON ((-158.046875 21.40625, -158.0078125 2...",{'USA': 1.0},{}
213,213,0.0,2.500696,0.000070,0.085259,0.000088,0.016480,0.000061,0.360667,9.998109e-15,0.393854,21615.452861,0.0,"POLYGON ((-158.046875 21.4375, -158.0078125 21...",{'USA': 1.0},{}
214,214,0.0,2.500696,0.000070,0.080900,0.000088,0.015638,0.000061,0.339166,1.005200e-14,0.371448,16469.949485,0.0,"POLYGON ((-158.046875 21.46875, -158.0078125 2...",{'USA': 1.0},{}


In [71]:
for index in range(len(fraction)):
    for country in Country:
        if country not in json.loads(fraction.iloc[index]['area_fraction'].replace('\'','\"')).keys():
            continue
        frac = json.loads(fraction.iloc[index]['area_fraction'].replace('\'','\"'))[country]
        fraction.iloc[index]['base_mort'][country] = frac * fraction.iloc[index]['TotalPop'] / Total_pop[country] * baseline[country]

In [73]:
fraction.to_csv('comparison/baseline_fraction_Hbar1.csv')

## Mortality rate per concentration comparison (Submit to cc part 2)

In [74]:
fraction = pd.read_csv('comparison/baseline_fraction_Hbar1.csv')

In [75]:
fraction['mort_per_con'] = [{} for i in range(len(fraction))]

In [83]:
for i in range(len(fraction)):
    for country in Country:
        if country not in json.loads(fraction.iloc[i]['base_mort'].replace('\'','\"')).keys():
            continue
        ratio = 1 - 1 / hazard_ratio_NCDLRI(fraction.iloc[i]['TotalPM25'])
        fraction.iloc[i]['mort_per_con'][country] = ratio * json.loads(fraction.iloc[i]['base_mort'].replace('\'','\"'))[country] / fraction.iloc[i]['TotalPM25']    

In [ ]:
fraction.to_csv('comparison/baseline_fraction_Hbar2.csv')